<a href="https://colab.research.google.com/github/jalevano/tfm_uoc_datascience/blob/main/03_Analisis_Fase_2G_Config.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
================================================================================
FASE 2G - CONFIGURACION COMPARTIDA
================================================================================

Trabajo Fin de Master - Evaluacion Comparativa de Tecnicas de Segmentacion
                        en Fotografia de Retrato

Autor: Jesus L.
Universidad: Universitat Oberta de Catalunya (UOC)
Master: Data Science
Fecha: Diciembre 2025

Descripcion:
    Configuracion centralizada para la Fase 2G (Rendimiento Computacional).
    Analisis de trade-offs entre calidad (IoU) y recursos (tiempo, memoria GPU).

USO EN GOOGLE COLAB:

    # PASO 1: Montar Drive
    from google.colab import drive
    drive.mount('/content/drive')

    # PASO 2: Cambiar al directorio de trabajo
    %cd /content/drive/MyDrive/TFM/3_Analisis/fase2g_rendimiento

    # PASO 3: Importar y ejecutar
    from analisis_fase_2g_config import *
    estado = inicializar_fase2g()

    from analisis_fase_2g_rendimiento import ejecutar_fase2g_completa
    resultados = ejecutar_fase2g_completa()

ESTRUCTURA DE SALIDA:

    fase2g_rendimiento/
    ├── analisis_fase_2g_config.py
    ├── analisis_fase_2g_rendimiento.py
    ├── datos/
    │   ├── rendimiento_por_config.csv
    │   ├── rendimiento_por_modelo.csv
    │   ├── eficiencia_por_config.csv
    │   └── frontera_pareto.csv
    ├── pareto_tiempo_iou.png/pdf
    ├── pareto_memoria_iou.png/pdf
    ├── barplot_tiempo_modelo.png/pdf
    ├── scatter_eficiencia.png/pdf
    └── REPORTE_FASE2G.md

================================================================================
"""

"\n================================================================================\nFASE 2G - CONFIGURACION COMPARTIDA\n================================================================================\n\nTrabajo Fin de Master - Evaluacion Comparativa de Tecnicas de Segmentacion\n                        en Fotografia de Retrato\n\nAutor: Jesus L.\nUniversidad: Universitat Oberta de Catalunya (UOC)\nMaster: Data Science\nFecha: Diciembre 2025\n\nDescripcion:\n    Configuracion centralizada para la Fase 2G (Rendimiento Computacional).\n    Analisis de trade-offs entre calidad (IoU) y recursos (tiempo, memoria GPU).\n\nUSO EN GOOGLE COLAB:\n    \n    # PASO 1: Montar Drive\n    from google.colab import drive\n    drive.mount('/content/drive')\n    \n    # PASO 2: Cambiar al directorio de trabajo\n    %cd /content/drive/MyDrive/TFM/3_Analisis/fase2g_rendimiento\n    \n    # PASO 3: Importar y ejecutar\n    from analisis_fase_2g_config import *\n    estado = inicializar_fase2g()\n    \n    

In [ ]:
# =============================================================================
# IMPORTACIONES
# =============================================================================

import os
import sys
import json
import logging
import warnings
from pathlib import Path
from datetime import datetime
from typing import Dict, List, Tuple, Optional, Any

import numpy as np
import pandas as pd
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import seaborn as sns

warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=UserWarning)

In [ ]:
# =============================================================================
# CONFIGURACION DE RUTAS
# =============================================================================

def montar_drive_colab():
    """
    Monta Google Drive en Colab si no esta montado.
    Llamar esta funcion al inicio del notebook.
    """
    try:
        from google.colab import drive
        if not os.path.exists('/content/drive/MyDrive'):
            print("Montando Google Drive...")
            drive.mount('/content/drive')
            print("Drive montado correctamente")
        else:
            print("Drive ya esta montado")
        return True
    except ImportError:
        print("No estamos en Google Colab")
        return False
    except Exception as e:
        print(f"Error montando Drive: {e}")
        return False

# Intentar montar Drive al importar el modulo
montar_drive_colab()


def detectar_entorno() -> Path:
    """Detecta el entorno de ejecucion y retorna la ruta base."""
    if os.path.exists('/content/drive/MyDrive'):
        return Path('/content/drive/MyDrive/TFM')
    elif os.path.exists('/mnt/user-data/uploads'):
        return Path('/mnt/user-data/uploads')
    else:
        return Path.cwd()


RUTA_BASE = detectar_entorno()

RUTAS = {
    # Datos de entrada
    'fase2a': RUTA_BASE / '3_Analisis' / 'fase2a_metricas',
    'fase2b': RUTA_BASE / '3_Analisis' / 'fase2b_correlaciones',
    'fase2d': RUTA_BASE / '3_Analisis' / 'fase2d_configuraciones',

    # Salida Fase 2G
    'fase2g': RUTA_BASE / '3_Analisis' / 'fase2g_rendimiento',
    'output_datos': RUTA_BASE / '3_Analisis' / 'fase2g_rendimiento' / 'datos',
}

ARCHIVOS = {
    'indice_maestro': RUTA_BASE / '3_Analisis' / 'fase1_integracion' / 'indice_maestro.json',
    'metricas_fusionadas': RUTAS['fase2b'] / 'metricas_fusionadas.csv',
    'metricas_alt': RUTAS['fase2a'] / 'todas_metricas.csv',
}

Drive ya esta montado


In [ ]:
# =============================================================================
# CONFIGURACION DE ESTILOS
# =============================================================================

STYLE_CONFIG = {
    'figure.dpi': 100,
    'figure.facecolor': 'white',
    'font.family': 'serif',
    'font.size': 10,
    'axes.titlesize': 12,
    'axes.labelsize': 10,
    'axes.titleweight': 'bold',
    'axes.spines.top': False,
    'axes.spines.right': False,
    'xtick.labelsize': 9,
    'ytick.labelsize': 9,
    'legend.fontsize': 9,
    'figure.titlesize': 14,
    'figure.titleweight': 'bold',
}

FIGSIZE = {
    'scatter': (10, 8),
    'barplot': (12, 6),
    'pareto': (11, 8),
    'panel': (14, 10),
}

EXPORT_CONFIG = {
    'dpi': 300,
    'formatos': ['png', 'pdf'],
    'bbox_inches': 'tight',
    'pad_inches': 0.1,
    'facecolor': 'white',
}

In [ ]:
# =============================================================================
# PALETAS DE COLORES
# =============================================================================

PALETA_MODELOS = {
    'YOLOv8': '#2ecc71',
    'OneFormer': '#3498db',
    'Mask2Former': '#9b59b6',
    'SAM2': '#e74c3c',
    'BodyPix': '#1abc9c',
}

# Mapeo de prefijos de configuracion a modelo base
MAPEO_CONFIG_MODELO = {
    'yolov8': 'YOLOv8',
    'oneformer': 'OneFormer',
    'mask2former': 'Mask2Former',
    'sam2': 'SAM2',
    'bodypix': 'BodyPix',
}

In [ ]:
# =============================================================================
# CONFIGURACION DE LOGGING
# =============================================================================

def configurar_logging(nivel: int = logging.INFO) -> logging.Logger:
    """Configura el sistema de logging."""
    logger = logging.getLogger('Fase2G')
    logger.setLevel(nivel)

    if not logger.handlers:
        handler = logging.StreamHandler()
        handler.setLevel(nivel)
        formato = logging.Formatter(
            '%(asctime)s | %(levelname)-8s | %(message)s',
            datefmt='%H:%M:%S'
        )
        handler.setFormatter(formato)
        logger.addHandler(handler)

    return logger


logger = configurar_logging()

In [ ]:
# =============================================================================
# FUNCIONES AUXILIARES
# =============================================================================

def crear_directorios() -> Dict[str, bool]:
    """Crea la estructura de directorios para Fase 2G."""
    directorios = [RUTAS['fase2g'], RUTAS['output_datos']]
    estado = {}

    for ruta in directorios:
        try:
            ruta.mkdir(parents=True, exist_ok=True)
            estado[str(ruta)] = True
        except Exception as e:
            estado[str(ruta)] = False
            logger.error(f"Error creando {ruta}: {e}")

    return estado


def cargar_indice_maestro() -> Optional[Dict]:
    """Carga el indice maestro del proyecto."""
    rutas = [
        ARCHIVOS['indice_maestro'],
        RUTA_BASE / 'indice_maestro.json',
    ]

    for ruta in rutas:
        if ruta and ruta.exists():
            try:
                with open(ruta, 'r', encoding='utf-8') as f:
                    indice = json.load(f)
                n_fotos = len([k for k in indice.keys() if not k.startswith('_metadata')])
                logger.info(f"Indice maestro cargado: {n_fotos} fotos desde {ruta.name}")
                return indice
            except Exception as e:
                logger.debug(f"Error con {ruta}: {e}")

    logger.error("No se pudo cargar el indice maestro")
    return None


def cargar_metricas() -> Optional[pd.DataFrame]:
    """Carga el CSV de metricas."""
    rutas = [
        ARCHIVOS['metricas_fusionadas'],
        ARCHIVOS['metricas_alt'],
        RUTA_BASE / '3_Analisis' / 'metricas_fusionadas.csv',
    ]

    for ruta in rutas:
        if ruta and ruta.exists():
            try:
                df = pd.read_csv(ruta)
                logger.info(f"Metricas cargadas: {len(df)} filas desde {ruta.name}")
                return df
            except Exception as e:
                logger.debug(f"Error con {ruta}: {e}")

    logger.error("No se pudo cargar metricas")
    return None


def obtener_modelo_base(codigo_config: str) -> str:
    """Extrae el nombre del modelo base desde el codigo de configuracion."""
    codigo_lower = codigo_config.lower()
    for prefijo, modelo in MAPEO_CONFIG_MODELO.items():
        if codigo_lower.startswith(prefijo):
            return modelo
    return 'Desconocido'


def aplicar_estilo():
    """Aplica el estilo academico a matplotlib."""
    plt.style.use('seaborn-v0_8-whitegrid')
    plt.rcParams.update(STYLE_CONFIG)


def guardar_figura(fig: plt.Figure, nombre: str,
                   ruta_base: Optional[Path] = None) -> List[Path]:
    """Guarda una figura en multiples formatos."""
    if ruta_base is None:
        ruta_base = RUTAS['fase2g']

    rutas_guardadas = []
    for formato in EXPORT_CONFIG['formatos']:
        ruta = ruta_base / f'{nombre}.{formato}'
        try:
            fig.savefig(
                ruta,
                dpi=EXPORT_CONFIG['dpi'],
                bbox_inches=EXPORT_CONFIG['bbox_inches'],
                pad_inches=EXPORT_CONFIG['pad_inches'],
                facecolor=EXPORT_CONFIG['facecolor']
            )
            rutas_guardadas.append(ruta)
        except Exception as e:
            logger.error(f"Error guardando {ruta}: {e}")

    return rutas_guardadas


def inicializar_fase2g() -> Dict[str, Any]:
    """Inicializa la Fase 2G."""
    logger.info("="*60)
    logger.info("INICIALIZANDO FASE 2G: RENDIMIENTO COMPUTACIONAL")
    logger.info("="*60)

    estado_dirs = crear_directorios()
    aplicar_estilo()

    # Verificar archivos
    indice = cargar_indice_maestro()
    metricas = cargar_metricas()

    exito = indice is not None and metricas is not None

    resultado = {
        'exito': exito,
        'ruta_base': str(RUTA_BASE),
        'directorios_creados': estado_dirs,
        'indice_cargado': indice is not None,
        'metricas_cargadas': metricas is not None,
    }

    if exito:
        logger.info("Inicializacion completada correctamente")
    else:
        logger.warning("Inicializacion con errores")

    return resultado

In [ ]:
# =============================================================================
# EXPORTS
# =============================================================================

__all__ = [
    'RUTA_BASE', 'RUTAS', 'ARCHIVOS',
    'STYLE_CONFIG', 'FIGSIZE', 'EXPORT_CONFIG',
    'PALETA_MODELOS', 'MAPEO_CONFIG_MODELO',
    'logger',
    'crear_directorios', 'cargar_indice_maestro', 'cargar_metricas',
    'obtener_modelo_base', 'aplicar_estilo', 'guardar_figura',
    'inicializar_fase2g',
]


if __name__ == '__main__':
    print("Test de configuracion Fase 2G")
    print(f"Ruta base: {RUTA_BASE}")
    estado = inicializar_fase2g()
    print(f"Estado: {'OK' if estado['exito'] else 'ERROR'}")

11:54:55 | INFO     | ============================================================
INFO:Fase2G:============================================================
11:54:55 | INFO     | INICIALIZANDO FASE 2G: RENDIMIENTO COMPUTACIONAL
INFO:Fase2G:INICIALIZANDO FASE 2G: RENDIMIENTO COMPUTACIONAL
11:54:55 | INFO     | ============================================================
INFO:Fase2G:============================================================


Test de configuracion Fase 2G
Ruta base: /content/drive/MyDrive/TFM


11:54:55 | INFO     | Indice maestro cargado: 0 fotos desde indice_maestro.json
INFO:Fase2G:Indice maestro cargado: 0 fotos desde indice_maestro.json
11:54:55 | INFO     | Metricas cargadas: 2360 filas desde metricas_fusionadas.csv
INFO:Fase2G:Metricas cargadas: 2360 filas desde metricas_fusionadas.csv
11:54:55 | INFO     | Inicializacion completada correctamente
INFO:Fase2G:Inicializacion completada correctamente


Estado: OK
